In [ ]:
# …existing code…
from timm.data.transforms_factory import create_transform
from torchvision import transforms

# Personnalisation du pipeline d'entraînement
train_transform = create_transform(
    input_size=INPUT_RESOLUTION,
    is_training=True,
    mean=model.config.mean,
    std=model.config.std,
    crop_mode=model.config.crop_mode,
    crop_pct=model.config.crop_pct,
    color_jitter=(0.3, 0.3, 0.3, 0.1),   # ajustement du jitter
    re_prob=0.2,                         # probabilité de RandomErasing
    re_count=1,
    re_mode='pixel',
    interpolation='bicubic'
)

# Ou un pipeline 100% torchvision si vous préférez un contrôle total
custom_transform = transforms.Compose([
    transforms.RandomResizedCrop(INPUT_RESOLUTION, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=model.config.mean, std=model.config.std),
])

# Personnalisation du pipeline d'entraînement
train_transform = create_transform(
    input_size=INPUT_RESOLUTION,
    is_training=True,
    mean=model.config.mean,
    std=model.config.std,
    crop_mode=model.config.crop_mode,
    crop_pct=model.config.crop_pct,
    color_jitter=(0.3, 0.3, 0.3, 0.1),   # ajustement du jitter
    re_prob=0.2,                         # probabilité de RandomErasing
    re_count=1,
    re_mode='pixel',
    interpolation='bicubic'
)

# Ou un pipeline 100% torchvision si vous préférez un contrôle total
from torchvision import transforms
custom_transform = transforms.Compose([
    transforms.RandomResizedCrop(INPUT_RESOLUTION, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=model.config.mean, std=model.config.std),
])


In [ ]:
model = AutoModel.from_pretrained("nvidia/MambaVision-L-21K", trust_remote_code=True)

# eval mode for inference
model.cuda().eval()

# prepare image for the model
url = 'http://images.cocodataset.org/val2017/000000020247.jpg'
image = Image.open(requests.get(url, stream=True).raw)
input_resolution = (3, 224, 224)  # MambaVision supports any input resolutions

transform = create_transform(input_size=input_resolution,
                             is_training=False,
                             mean=model.config.mean,
                             std=model.config.std,
                             crop_mode=model.config.crop_mode,
                             crop_pct=model.config.crop_pct)
inputs = transform(image).unsqueeze(0).cuda()
# model inference
out_avg_pool, features = model(inputs)
print("Size of the averaged pool features:", out_avg_pool.size())  # torch.Size([1, 768])
print("Number of stages in extracted features:", len(features)) # 4 stages
print("Size of extracted features in stage 1:", features[0].size()) # torch.Size([1, 96, 56, 56])
print("Size of extracted features in stage 4:", features[3].size()) # torch.Size([1, 768, 7, 7])